In [3]:
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import pandas as pd 

d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# loader = CSVLoader(file_path=r"D:\Internship\EnterpriseKnowledgeBased\output\sorted_data_20240718_160948_csv.csv")

# document = loader.load()
# docker run -p 6333:6333 -v D:/Internship/EnterpriseKnowledgeBased/qdrant/storage qdrant/qdrant
#docker run -p 6333:6333 -v D:/Internship/EnterpriseKnowledgeBased:/qdrant/data qdrant/qdrant

df = pd.read_csv(r'D:\Internship\EnterpriseKnowledgeBased\output\sorted_data_20240718_160948_csv.csv')
questions_answers = df[['Question', 'Answer']]
# texts = list(questions_answers.itertuples(index=False, name=None))
texts = [f"{row['Question']}  {row['Answer']}" for _, row in questions_answers.iterrows()]
texts

['What is City Touch?  City Touch is a simple, hassle-free and secure digital banking service available for all City Bank customers.',
 'Why should I choose City Touch?  City Touch is easy to register online, free to use, provides instant access to up-to-date account information, ensures safe and easy transactions, fast processing, convenient online banking from home or office, 24-hour access, worldwide accessibility, no special software required, and offers a range of online banking services for better financial management.',
 'Do I have to pay extra for this service?  There is no extra charge for this service.',
 'Is the service available 24-hours?  Yes. This service is available 24-hours a day, 7 days a week.',
 'Can I access this from overseas?  Yes. As long as you can access the internet with the recommended internet browsers, you will be able to access this service.',
 'Can I see a summary of my accounts?  Yes. You can see a summary of your account profile at the Personal Homepag

In [5]:
questions_answers 


,Question,Answer
0,What is City Touch?,"City Touch is a simple, hassle-free and secure..."
1,Why should I choose City Touch?,"City Touch is easy to register online, free to..."
2,Do I have to pay extra for this service?,There is no extra charge for this service.
3,Is the service available 24-hours?,"Yes. This service is available 24-hours a day,..."
4,Can I access this from overseas?,Yes. As long as you can access the internet wi...
5,Can I see a summary of my accounts?,Yes. You can see a summary of your account pro...
6,Can I view the transactions belonging to all m...,"Yes, you can view the transactions belonging t..."
7,What is the available duration for my transact...,You can view your transactional history up to ...
8,Can I download and save my account information...,Yes. You can select your latest transaction or...
9,Can I get confirmation of my transaction?,"Yes, you can get transaction alerts."


In [6]:
# document

In [7]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)

In [8]:
# text  = text_splitter.split_documents(document)

In [9]:
# Load the model
model_name = "all-mpnet-base-v2"
model = SentenceTransformer(model_name)

# Generate embeddings for the texts
embeddings = model.encode(texts, convert_to_tensor=False)

# Ensure embeddings are lists of floats
text_embeddings = [embedding.tolist() for embedding in embeddings]

In [10]:
# model_name = "BAAI/bge-large-en"
# model_kwargs = {'device': 'cpu'}
# # encode_kwargs = {'normalise_embeddings':False}

# embeddings_model = HuggingFaceBgeEmbeddings(model_name = model_name,model_kwargs=model_kwargs)
# # Generate embeddings
# embeddings = embeddings_model.embed_documents(texts)
# embeddings = [embedding.tolist() if not isinstance(embedding, list) else embedding for embedding in embeddings]

In [11]:
from qdrant_client.models import PointStruct,VectorParams,Distance 
from qdrant_client import QdrantClient

url = 'http://localhost:6333'
collection_name  = "faq_db"

client = QdrantClient(url)
client.create_collection(
    collection_name = collection_name,
    vectors_config=VectorParams(
        size=768,
        distance=Distance.COSINE,
    ),
)

points = [
    PointStruct(
        id=i,
        vector=embedding,
        payload={"text": texts[i]}
    )
    # for i, (text, embedding) in enumerate(zip(texts, embeddings))
    for i, embedding in enumerate(text_embeddings)
]

# Insert points into the collection
client.upsert(collection_name=collection_name, points=points)


# vector_store = Qdrant.from_documents(
#     texts,
#     embeddings,
#     url = url,
#     prefer_grpc = False,
#     collection_name = collection_name
# )


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `faq_db` already exists!"},"time":0.431143417}'

In [ ]:
collections = client.get_collections()
print("Collections in Qdrant:")
for collection in collections:
    print(collection)

Collections in Qdrant:
('collections', [CollectionDescription(name='faq_db')])


In [ ]:
#searching for similar results for the query
def search_similar_texts(query,top_k=5):
    # Generate embeddings for the texts
    query_embeddings = model.encode(query, convert_to_tensor=False)

    # Ensure embeddings are lists of floats
    query_embedding = [query_embedding.tolist() for query_embedding in query_embeddings]

    search_result = client.search(
        collection_name = collection_name,
        query_vector = query_embedding,
        limit= top_k
    )
    results = [{"text": res.payload["text"], "score": res.score} for res in search_result]
    return results

In [ ]:
#searching for the dissimliar text for the query
def find_dissimilar_text(query,top_k = 5):
    # Generate embeddings for the texts
    query_embeddings = model.encode(query, convert_to_tensor=False)
   # Ensure embeddings are lists of floats
    query_embedding = [query_embedding.tolist() for query_embedding in query_embeddings]

    search_result = client.search(
        collection_name = collection_name,
        query_vector = query_embedding,
        limit= top_k,
        score_threshold=0.0
    ) 
    results = sorted(
        [{"text": res.payload["text"], "score": res.score} for res in search_result],
        key=lambda x: x["score"]
    )[::-1]  # Reversing to get the most dissimilar
    return results


In [ ]:
# Example usage
query = "What is City Touch?"
results = search_similar_texts(query)
results


[{'text': 'What is City Touch?  City Touch is a simple, hassle-free and secure digital banking service available for all City Bank customers.',
  'score': 0.7931231},
 {'text': 'Why should I choose City Touch?  City Touch is easy to register online, free to use, provides instant access to up-to-date account information, ensures safe and easy transactions, fast processing, convenient online banking from home or office, 24-hour access, worldwide accessibility, no special software required, and offers a range of online banking services for better financial management.',
  'score': 0.59455013},
 {'text': "How do I apply for this service?  Go to the City Bank corporate website home page, click on the link 'Download City Touch Form'. A PDF application form will be downloaded. Take a printout, fill out the form and submit it to the nearest branch of City Bank.",
  'score': 0.47516388},
 {'text': 'How Islamic Banking customer will enroll into Citytouch?  Islamic Banking customer can log on to 

In [ ]:
dissimilar_result =find_dissimilar_text(query)
dissimilar_result

[{'text': 'What is City Touch?  City Touch is a simple, hassle-free and secure digital banking service available for all City Bank customers.',
  'score': 0.7931231},
 {'text': 'Why should I choose City Touch?  City Touch is easy to register online, free to use, provides instant access to up-to-date account information, ensures safe and easy transactions, fast processing, convenient online banking from home or office, 24-hour access, worldwide accessibility, no special software required, and offers a range of online banking services for better financial management.',
  'score': 0.59455013},
 {'text': "How do I apply for this service?  Go to the City Bank corporate website home page, click on the link 'Download City Touch Form'. A PDF application form will be downloaded. Take a printout, fill out the form and submit it to the nearest branch of City Bank.",
  'score': 0.47516388},
 {'text': 'How Islamic Banking customer will enroll into Citytouch?  Islamic Banking customer can log on to 